In [1]:
import cv2
import numpy as np
from scipy.ndimage import filters

In [7]:
def stylize_image(input_image_path, output_image_path, gaussian_kernel_size=5, bilateral_sigma_s=60, bilateral_sigma_r=0.07):
    # Read the input image
    img = cv2.imread(input_image_path)

    # Apply Gaussian blur to reduce image details
    blurred_img = cv2.GaussianBlur(img, (gaussian_kernel_size, gaussian_kernel_size), 0)

    # Apply Bilateral filter for region smoothing
    stylized_img = cv2.bilateralFilter(blurred_img, -1, bilateral_sigma_s, bilateral_sigma_r)

    # Save the stylized image
    cv2.imwrite(output_image_path, stylized_img)

if __name__ == "__main__":
    input_path = "hoja.png"
    output_path = "z.png"

    stylize_image(input_path, output_path)

In [8]:


def coherence_enhancing_diffusion(input_image_path, output_image_path, iterations=10, gamma=10):
    # Read the input image
    img = cv2.imread(input_image_path, cv2.IMREAD_GRAYSCALE)

    # Convert to float32 for the diffusion operation
    img_float32 = img.astype(np.float32)

    for _ in range(iterations):
        # Compute the gradient of the image
        gradient_x = cv2.Sobel(img_float32, cv2.CV_64F, 1, 0, ksize=3)
        gradient_y = cv2.Sobel(img_float32, cv2.CV_64F, 0, 1, ksize=3)
        gradient_magnitude = cv2.magnitude(gradient_x, gradient_y)

        # Compute the divergence of the diffusion tensor multiplied by the gradient
        divergence = np.exp(-gamma * gradient_magnitude) * (gradient_x * gradient_x + gradient_y * gradient_y)
        
        # Update the image using the divergence
        img_float32 += divergence

    # Convert back to uint8 for saving the image
    output_img = np.clip(img_float32, 0, 255).astype(np.uint8)

    # Save the result
    cv2.imwrite(output_image_path, output_img)

if __name__ == "__main__":
    input_path = "z.png"
    output_path = "x.png"

    coherence_enhancing_diffusion(input_path, output_path)

In [9]:
def enhance_spatial_hierarchy(input_image_path, output_image_path, direction='horizontal', scale_factor=1.0):
    # Read the input image
    img = cv2.imread(input_image_path, cv2.IMREAD_GRAYSCALE)

    # Compute the second derivative along the specified direction
    if direction == 'horizontal':
        I_uu = cv2.Laplacian(img, cv2.CV_64F, ksize=3)
    elif direction == 'vertical':
        I_uu = cv2.Laplacian(img, cv2.CV_64F, ksize=3).T
    else:
        raise ValueError("Invalid direction. Choose 'horizontal' or 'vertical'.")

    # Compute the magnitude of the gradient of the image
    gradient_magnitude = cv2.magnitude(cv2.Sobel(img, cv2.CV_64F, 1, 0, ksize=3), cv2.Sobel(img, cv2.CV_64F, 0, 1, ksize=3))

    # Perform the edge-enhancing operation
    result = -np.sign(I_uu) * gradient_magnitude

    # Scale the result if needed
    result *= scale_factor

    # Clip values to the valid range and convert back to uint8 for saving
    output_img = np.clip(result, 0, 255).astype(np.uint8)

    # Save the result
    cv2.imwrite(output_image_path, output_img)

if __name__ == "__main__":
    input_path = "x.png"
    output_path = "y.png"

    enhance_spatial_hierarchy(input_path, output_path, direction='horizontal', scale_factor=1.0)

In [7]:
def stylize_and_detect_borders(input_image_path, output_image_path):
    # Step 1: Gaussian Filtering
    img = cv2.imread(input_image_path)
    gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    blurred_img = cv2.GaussianBlur(gray_img, (5, 5), 0)

    # Step 2: Bilateral Filtering
    bilateral_img = cv2.bilateralFilter(blurred_img, -1, 60, 0.05)

    # Step 3: Coherence-Enhancing Diffusion
    ced_img = coherence_enhancing_diffusion(bilateral_img, iterations=10, gamma=10)

    x_img = enhance_spatial_hierarchy(ced_img, direction='horizontal', scale_factor=1.0)


    # Save the result
    cv2.imwrite(output_image_path, x_img)

def coherence_enhancing_diffusion(img, iterations=10, gamma=10):
    # Convert to float32 for the diffusion operation
    img_float32 = img.astype(np.float32)

    for _ in range(iterations):
        # Compute the gradient of the image
        gradient_x = cv2.Sobel(img_float32, cv2.CV_64F, 1, 0, ksize=3)
        gradient_y = cv2.Sobel(img_float32, cv2.CV_64F, 0, 1, ksize=3)
        gradient_magnitude = cv2.magnitude(gradient_x, gradient_y)

        # Compute the divergence of the diffusion tensor multiplied by the gradient
        divergence = np.exp(-gamma * gradient_magnitude) * (gradient_x * gradient_x + gradient_y * gradient_y)
        
        # Update the image using the divergence
        img_float32 += divergence

    # Convert back to uint8 for further processing
    output_img = np.clip(img_float32, 0, 255).astype(np.uint8)

    return output_img

def enhance_spatial_hierarchy(input_image_path, direction='horizontal', scale_factor=1.0):
    # Read the input image
    img = input_image_path

    # Compute the second derivative along the specified direction
    if direction == 'horizontal':
        I_uu = cv2.Laplacian(img, cv2.CV_64F, ksize=3)
    elif direction == 'vertical':
        I_uu = cv2.Laplacian(img, cv2.CV_64F, ksize=3).T
    else:
        raise ValueError("Invalid direction. Choose 'horizontal' or 'vertical'.")

    # Compute the magnitude of the gradient of the image
    gradient_magnitude = cv2.magnitude(cv2.Sobel(img, cv2.CV_64F, 1, 0, ksize=3), cv2.Sobel(img, cv2.CV_64F, 0, 1, ksize=3))

    # Perform the edge-enhancing operation
    result = -np.sign(I_uu) * gradient_magnitude

    # Scale the result if needed
    result *= scale_factor

    # Clip values to the valid range and convert back to uint8 for saving
    output_img = np.clip(result, 0, 255).astype(np.uint8)

    return output_img

if __name__ == "__main__":
    input_path = "hoja.png"
    output_path = "b.png"

    stylize_and_detect_borders(input_path, output_path)